Igual que el v2

In [1]:
#Define the data available for the model
metal_data = {
    "unit":"gram",
    "currency": "USD",
    "prices": {
        "gold": 88.1553,
        "silver": 1.0523,
        "platinum": 32.169,
        "palladium": 35.8252,
        "copper": 0.0098,
        "aluminum": 0.0026,
        "lead": 0.0021,
        "nickel": 0.0159,
        "zinc": 0.0031,
    }
}

In [2]:
from langchain_core.tools import tool
import requests

# Define the tools for the agent to use, it is necessary to specify that each function is a tool
@tool
def get_metal_price(metal_name: str) -> str:
    """Fetches the current per gram in USD price of the specified metal.

    Args:
        metal_name : The name of the metal (e.g., 'gold', 'silver', 'platinum').

    Returns:
        float: The current price of the metal in dollars per gram.

    Raises:
        KeyError: If the specified metal is not found in the data source.
    """
    try:
        metal_name = metal_name.lower().strip()
        prices = metal_data["prices"]
        currency = metal_data["currency"]
        unit=metal_data["unit"]
        if metal_name not in prices:
            raise KeyError(
                f"Metal {metal_name} not found. Available metals: {', '.join(metal_data['prices'].keys())}"
            )
        price=prices[metal_name]
        return f"The current price of {metal_name} is {price} {currency} per {unit}."
    except Exception as e:
        raise Exception(f"Error fetching metal price: {str(e)}")
    
@tool    
def get_currency_exchange(base: str, target: str) -> str:
    """
    Returns the exchange rate from base currency to target currency.

    Args:
        base (str): The base currency (e.g., 'USD').
        target (str): The target currency (e.g., 'EUR').

    Returns:
        str: A human-readable string showing the exchange rate,
             or an error message if the pair is not found.
    """
    fake_rates = {
        ("usd", "eur"): 0.8,
        ("eur", "usd"): 1.8,
        ("usd", "gbp"): 0.7
    }
    rate = fake_rates.get((base.lower(), target.lower()))
    if rate is None:
        return f"No exchange rate found for {base.upper()} to {target.upper()}"
    return  f"{base.upper()} = {rate} {target.upper()}"

from langchain_core.tools import tool
import requests

@tool
def get_summary_of(person: str) -> str:
    """
    Returns a short summary of a famous person from Wikipedia.
    """
    try:
        title = person.replace(" ", "_")
        url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title}"
        response = requests.get(url, timeout=10,verify=False)
        response.raise_for_status()
        data = response.json()

        return data.get("extract", "No summary available.")
    
    except Exception as e:
        return f"Error: {e}"


@tool
def get_current_president_of(country: str) -> str:
    """
    Returns the name of the current president of a given country using Wikidata.
    """
    try:
        # Step 1: Get Wikidata ID for the country
        search_url = "https://www.wikidata.org/w/api.php"
        search_params = {
            "action": "wbsearchentities",
            "search": country,
            "language": "en",
            "format": "json",
            "type": "item"
        }
        search_resp = requests.get(search_url, params=search_params, timeout=10,verify=False)
        search_resp.raise_for_status()
        search_data = search_resp.json()
        entity_id = search_data["search"][0]["id"]

        # Step 2: Query president (P35) of the country entity
        sparql_url = "https://query.wikidata.org/sparql"
        query = f"""
        SELECT ?presidentLabel WHERE {{
          wd:{entity_id} wdt:P35 ?president.
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        """
        headers = {"Accept": "application/sparql-results+json"}
        sparql_resp = requests.get(sparql_url, params={"query": query}, headers=headers, timeout=10,verify=False)
        sparql_resp.raise_for_status()
        result = sparql_resp.json()
        bindings = result["results"]["bindings"]

        if not bindings:
            return "President not found."
        return bindings[0]["presidentLabel"]["value"]

    except Exception as e:
        return f"Error: {e}"


In [3]:
from langchain_ollama.chat_models import ChatOllama
from langgraph.prebuilt import create_react_agent

# Instanciamos el modelo LLM local usando Ollama (Llama 3.2)
llm = ChatOllama(
    model="llama3.2",   # Usamos el modelo Llama 3.2 local
    temperature=0
)

# Vinculamos nuestra herramienta al LLM
tools = [get_metal_price,get_currency_exchange,get_current_president_of,get_summary_of]
llm_with_tools = llm.bind_tools(tools)

agent = create_react_agent(
    model=llm_with_tools,
    tools=[get_metal_price, get_currency_exchange,get_current_president_of,get_summary_of],
    prompt = """
You are a ReAct agent.

You do NOT know the answer to any question before using tools.

For questions about metal prices like "What is the price of METAL in CUR?":
1. Call the tool `get_metal_price` with the metal name (e.g., "gold").
2. Then call `get_currency_exchange` with base='USD' and target=CUR.
3. Combine both results in a final response with the price converted.

For questions like "Who is the president of COUNTRY?":
1. Call `get_current_president_of` with the country name.
2. Use the result directly as the final answer.
3. If the result is not a person’s name or indicates failure, respond with:
   "I could not find that information in the retrieved source."

For questions like "Give me a summary of PERSON":
1. Call `get_summary_of` with the person's name.
2. Return the extract from the tool as your final answer.

Never invent answers. Use only information provided by the tools.
"""

)


In [4]:
from langgraph.prebuilt import ToolNode
from langgraph.graph import END
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
from langchain.schema.messages import AIMessage,ToolMessage,HumanMessage

# 1) State
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: GraphState):
    result = agent.invoke({"messages": state["messages"]})
    new_msgs = result["messages"]
    return {"messages": state["messages"] + new_msgs}

In [5]:
from langgraph.graph import START, END, StateGraph
def verify_next(state: GraphState):
    last = state["messages"][-1]
    return END if isinstance(last, AIMessage) and not getattr(last, "tool_calls", None) else "assistant"

builder = StateGraph(GraphState)
builder.add_node("assistant", assistant)

builder.add_edge(START, "assistant")                         # arranque
builder.add_edge("assistant", END)                          # siempre pasar por verify

react_graph = builder.compile()
from IPython.display import Image, display
#display(Image(react_graph.get_graph(xray=True).draw_mermaid_png()))

In [6]:
from langchain.schema.messages import AIMessage,ToolMessage,HumanMessage
from copy import deepcopy
import json
import uuid

def normalize_args(args):
    return {k.lower(): str(v).lower() for k, v in args.items()}

def fix_tool_calls_for_openai_format(messages):
    final_messages = []
    tool_message_buffer = {}
    used_tool_call_ids = set()

    # Indexar ToolMessages por tool_call_id
    for msg in messages:
        if isinstance(msg, ToolMessage):
            tool_message_buffer[msg.tool_call_id] = msg

    for msg in messages:
        if isinstance(msg, HumanMessage):
            final_messages.append(msg)

        elif isinstance(msg, AIMessage) and msg.tool_calls and len(msg.tool_calls) > 1:
            for tool_call in msg.tool_calls:
                # Normalizar los args aquí
                norm_args = normalize_args(tool_call["args"])

                new_msg = deepcopy(msg)
                new_msg.tool_calls = [{
                    "name": tool_call["name"],
                    "args": norm_args,
                    "id": tool_call.get("id", f"call_{uuid.uuid4().hex[:24]}"),
                    "type": "tool_call"
                }]
                new_msg.additional_kwargs["tool_calls"] = [{
                    "id": tool_call.get("id", f"call_{uuid.uuid4().hex[:24]}"),
                    "type": "function",
                    "function": {
                        "name": tool_call["name"],
                        "arguments": json.dumps(norm_args)
                    }
                }]
                final_messages.append(new_msg)

                tool_msg = tool_message_buffer.get(tool_call["id"])
                if tool_msg:
                    final_messages.append(tool_msg)
                    used_tool_call_ids.add(tool_call["id"])

        elif isinstance(msg, AIMessage) and msg.tool_calls:
            tool_call = msg.tool_calls[0]
            norm_args = normalize_args(tool_call["args"])

            msg.tool_calls[0]["args"] = norm_args
            msg.additional_kwargs["tool_calls"] = [{
                "id": tool_call.get("id", f"call_{uuid.uuid4().hex[:24]}"),
                "type": "function",
                "function": {
                    "name": tool_call["name"],
                    "arguments": json.dumps(norm_args)
                }
            }]
            final_messages.append(msg)

        elif isinstance(msg, AIMessage):
            final_messages.append(msg)

        elif isinstance(msg, ToolMessage):
            if msg.tool_call_id not in used_tool_call_ids:
                final_messages.append(msg)

    return final_messages


In [7]:
result = react_graph.invoke({"messages": [HumanMessage(content="Who is the president of France?")]})
#result = react_graph.invoke({"messages": [HumanMessage(content="When did Emmanuel Macron become president of France?")]})
result

c:\Users\eguzk\anaconda3\envs\TFM\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\eguzk\anaconda3\envs\TFM\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'query.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'messages': [HumanMessage(content='Who is the president of France?', additional_kwargs={}, response_metadata={}, id='5577fa39-5638-489d-a0bf-349b8eb3f6f5'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-06-03T13:38:34.8193773Z', 'done': True, 'done_reason': 'stop', 'total_duration': 19896977000, 'load_duration': 3497870800, 'prompt_eval_count': 683, 'prompt_eval_duration': 14451717800, 'eval_count': 20, 'eval_duration': 1937311700, 'model_name': 'llama3.2'}, id='run--fd22aeee-ad82-4f7c-8007-3bd0515d3284-0', tool_calls=[{'name': 'get_current_president_of', 'args': {'country': 'France'}, 'id': 'cb7b680c-9687-4605-8dd2-5dd8f2e13dcb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 683, 'output_tokens': 20, 'total_tokens': 703}),
  ToolMessage(content='Emmanuel Macron', name='get_current_president_of', id='fbde5d7a-adde-438d-8873-89ef83c7d22a', tool_call_id='cb7b680c-9687-4605-8dd2-5dd8f2e13dcb'),
  AIMessage(content='The 

In [8]:
ragas_trace=fix_tool_calls_for_openai_format(result["messages"])
ragas_trace

[HumanMessage(content='Who is the president of France?', additional_kwargs={}, response_metadata={}, id='5577fa39-5638-489d-a0bf-349b8eb3f6f5'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'cb7b680c-9687-4605-8dd2-5dd8f2e13dcb', 'type': 'function', 'function': {'name': 'get_current_president_of', 'arguments': '{"country": "france"}'}}]}, response_metadata={'model': 'llama3.2', 'created_at': '2025-06-03T13:38:34.8193773Z', 'done': True, 'done_reason': 'stop', 'total_duration': 19896977000, 'load_duration': 3497870800, 'prompt_eval_count': 683, 'prompt_eval_duration': 14451717800, 'eval_count': 20, 'eval_duration': 1937311700, 'model_name': 'llama3.2'}, id='run--fd22aeee-ad82-4f7c-8007-3bd0515d3284-0', tool_calls=[{'name': 'get_current_president_of', 'args': {'country': 'france'}, 'id': 'cb7b680c-9687-4605-8dd2-5dd8f2e13dcb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 683, 'output_tokens': 20, 'total_tokens': 703}),
 ToolMessage(content='Emmanuel Macron', 

In [9]:
result = react_graph.invoke({"messages": [HumanMessage(content="Describe the president of France?")]})
result

c:\Users\eguzk\anaconda3\envs\TFM\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\eguzk\anaconda3\envs\TFM\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'query.wikidata.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{'messages': [HumanMessage(content='Describe the president of France?', additional_kwargs={}, response_metadata={}, id='c044dd04-39e0-45c4-b44f-a90957a1bc7e'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-06-03T13:38:51.2040594Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12396957200, 'load_duration': 25585800, 'prompt_eval_count': 682, 'prompt_eval_duration': 10358418100, 'eval_count': 20, 'eval_duration': 2011889600, 'model_name': 'llama3.2'}, id='run--5946622d-bea0-4f5b-8004-a386204171e9-0', tool_calls=[{'name': 'get_current_president_of', 'args': {'country': 'France'}, 'id': '244bdc9d-defa-4a93-9fca-5562780e5dfd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 682, 'output_tokens': 20, 'total_tokens': 702}),
  ToolMessage(content='Emmanuel Macron', name='get_current_president_of', id='abad4bcc-9357-4630-adb3-5098226a0693', tool_call_id='244bdc9d-defa-4a93-9fca-5562780e5dfd'),
  AIMessage(content='The 

In [10]:
ragas_trace=fix_tool_calls_for_openai_format(result["messages"])
ragas_trace

[HumanMessage(content='Describe the president of France?', additional_kwargs={}, response_metadata={}, id='c044dd04-39e0-45c4-b44f-a90957a1bc7e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '244bdc9d-defa-4a93-9fca-5562780e5dfd', 'type': 'function', 'function': {'name': 'get_current_president_of', 'arguments': '{"country": "france"}'}}]}, response_metadata={'model': 'llama3.2', 'created_at': '2025-06-03T13:38:51.2040594Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12396957200, 'load_duration': 25585800, 'prompt_eval_count': 682, 'prompt_eval_duration': 10358418100, 'eval_count': 20, 'eval_duration': 2011889600, 'model_name': 'llama3.2'}, id='run--5946622d-bea0-4f5b-8004-a386204171e9-0', tool_calls=[{'name': 'get_current_president_of', 'args': {'country': 'france'}, 'id': '244bdc9d-defa-4a93-9fca-5562780e5dfd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 682, 'output_tokens': 20, 'total_tokens': 702}),
 ToolMessage(content='Emmanuel Macron', 

In [12]:
from ragas.dataset_schema import SingleTurnSample
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# convierte los mensajes de LangChain a SingleTurnSample
def lc_to_ragas_sample(lc_msgs, reference: str) -> SingleTurnSample:
    question = next(m.content for m in lc_msgs if isinstance(m, HumanMessage))
    answer   = next(
        m.content for m in reversed(lc_msgs)
        if isinstance(m, AIMessage) and not getattr(m, "tool_calls", None)
    )
    contexts = [m.content for m in lc_msgs if isinstance(m, ToolMessage)]

    return SingleTurnSample(
        user_input=question,
        response=answer,
        retrieved_contexts=contexts,
        reference= reference
           )

# historial de tu agente (por ejemplo result["messages"])
ragas_sample = lc_to_ragas_sample(result["messages"],"Emmanuel Macron")
ragas_sample

c:\Users\eguzk\anaconda3\envs\TFM\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


SingleTurnSample(user_input='Describe the president of France?', retrieved_contexts=['Emmanuel Macron'], reference_contexts=None, response='The current President of France is Emmanuel Macron.', multi_responses=None, reference='Emmanuel Macron', rubrics=None)

In [13]:
from ragas import EvaluationDataset

ragas_dataset = EvaluationDataset.from_list(
    [ragas_sample.model_dump()]          # ← convertir a dict
)
ragas_dataset

EvaluationDataset(features=['user_input', 'retrieved_contexts', 'response', 'reference'], len=1)

In [ ]:
from ragas.evaluation import evaluate
from ragas.metrics import   AnswerRelevancy,ContextPrecision,Faithfulness
from ragas.llms import LangchainLLMWrapper
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import Ollama
from ragas import RunConfig


# Cargar modelo de embeddings open-source
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Envolver el modelo local
local_llm = Ollama(model="mistral", temperature=0,timeout=60000)
wrapped_llm = LangchainLLMWrapper(local_llm)

metrics = [AnswerRelevancy(),Faithfulness(), ContextPrecision()]
#metrics = [Faithfulness()]


#faithfulness needs ['retrieved_contexts']
#ContextPrecision needs reference

run_cfg = RunConfig(
    timeout=3000,      # 5 minutos por job
    max_retries=1,    # no reintentes eternamente si falla
    max_workers=1     # todo secuencial → nada de procesos paralelos que saturen la RAM/CPU
)

# Ejecutar la evaluación
result_eval = evaluate(
    ragas_dataset,
    metrics=metrics,
    llm=wrapped_llm,
    embeddings=embeddings,
    batch_size=1,
    raise_exceptions=True,   # para ver el trace completo si algo peta
    run_config=run_cfg
)
#0.954384	1.0	1.0
# Visualizar resultados
result_eval.to_pandas()



Evaluating: 100%|██████████| 3/3 [04:06<00:00, 82.29s/it]


,user_input,retrieved_contexts,response,reference,answer_relevancy,faithfulness,context_precision
0,Describe the president of France?,[Emmanuel Macron],The current President of France is Emmanuel Ma...,Emmanuel Macron,0.84697,1.0,1.0
